# Практика Text classification using CNN

Берем отзывы за лето (из архива с материалами или предыдущего занятия)

Учим conv сеть для классификации
Рассмотреть 2-а варианта сеточек:
Инициализировать слой tf.keras.layers.Embedding по умолчанию (ну то есть вам ничего не делать с весами)
Инициализировать tf.keras.layers.Embedding предобученными векторами взять к примеру с https://rusvectores.org/ru/
Сравнить две архитектуры с предобученными весами и когда tf.keras.layers.Embedding обучается сразу со всей сеточкой, что получилось лучше


In [1]:
pip install pymorphy3


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 599.2 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 55.6 MB/s eta 0:00:00


In [2]:
pip install stop_words

  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=ca545dab1c9f6d49c4ad374499f1596dfe8a715d97e9e9b0b7a6fdb475451858
  Stored in directory: /root/.cache/pip/wheels/d0/1a/23/f12552a50cb09bcc1694a5ebb6c2cd5f2a0311de2b8c3d9a89
Successfully built stop_words


In [3]:
import pandas as pd

from string import punctuation
from stop_words import get_stop_words
from pymorphy3 import MorphAnalyzer
import re

import nltk
from nltk.tokenize import word_tokenize
nltk.download("punkt")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
pd.set_option('max_colwidth', 1200)
pd.set_option('display.width', 1200)
pd.set_option('display.width', 100)
pd.set_option('display.expand_frame_repr', True)
pd.set_option('styler.latex.multicol_align',  'naive-l')
pd.set_option('display.colheader_justify', 'left')

In [5]:
max_words = 900
max_len = 60
num_classes = 2

# Training
epochs = 30
batch_size = 1024
print_batch_n = 200

In [6]:
df = pd.read_excel("/content/отзывы за лето.xls")
df.head(30)

,Rating,Content,Date
0,5,It just works!,2017-08-14
1,4,"В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в телефоне,приходится пользоваться в ограниченном режиме",2017-08-14
2,5,Отлично все,2017-08-14
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14
4,5,"Очень удобно, работает быстро.",2017-08-14
5,5,Всё удобно норм 👍👍👍,2017-08-14
6,5,Очень удобное приложение.,2017-08-14
7,5,Все устраивает,2017-08-14
8,5,"У меня работает все четко. В отличии от банкоматов, которые вечно зависают и тупят",2017-08-14
9,5,Очень все хорошо👍,2017-08-14


In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:

df.loc[df['Rating']>3, 'class'] = 1
df.loc[df['Rating']<3, 'class'] = 0

In [9]:
df.dropna(inplace=True)

In [10]:
df.isna().sum()

Rating     0
Content    0
Date       0
class      0
dtype: int64

### Предобработка

In [11]:
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
morpher = MorphAnalyzer()

def preprocess_text(txt):

    txt = str(txt)
    #txt = bytes(txt, encoding='utf8')
    #txt = txt.decode("unicode_escape")
    txt = re.sub(r'http\S+\s*', '', txt) # удалит URL
    txt = re.sub(r'RT|cc', '', txt) # удалит RT и cc
    txt = re.sub(r'ха\S+|ax\S+', ' смешно ', txt)
    txt = re.sub(r'\S+\)\S+|\S+:D\S+|D\S+', ' смешно ', txt)
    txt = re.sub(r',|!', ' ', txt) # обход если слова разделяют только запятой или ! и пробела нет
    txt = re.sub(r'приложение|антивирус', ' антивирус ', txt)
    txt = re.sub('очень', ' ', txt)
    txt = re.sub(r'\S+\(\S+', ' грустно ', txt)
    #txt = " ".join(c for c in txt if c not in exclude)
    txt = txt.lower()
    txt = re.sub('\s+', ' ', txt) # заменит пробельные символы на 1 пробел
    txt = "".join(c for c in txt if c.isalpha() or c == ' ') # удалит все что не буква

    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if len(word) > 1] #word not in sw and
    return " ".join(txt)

df['Content_1'] = df['Content'].apply(preprocess_text)


In [12]:
df.head(40)

,Rating,Content,Date,class,Content_1
0,5,It just works!,2017-08-14,1.0,it just works
1,4,"В целом удобноное приложение...из минусов хотят слишком большой доступ к персональным данным в телефоне,приходится пользоваться в ограниченном режиме",2017-08-14,1.0,целое удобноной антивирус из минус хотеть слишком большой доступ персональный данные телефон приходиться пользоваться ограниченный режим
2,5,Отлично все,2017-08-14,1.0,отлично всё
3,5,Стал зависать на 1% работы антивируса. Дальше никуда. Ранее больше года пользовался нормально.,2017-08-14,1.0,стать зависать на работа антивирус далёкий никуда ранее большой год пользоваться нормально
4,5,"Очень удобно, работает быстро.",2017-08-14,1.0,очень удобно работать быстро
5,5,Всё удобно норм 👍👍👍,2017-08-14,1.0,всё удобно норма
6,5,Очень удобное приложение.,2017-08-14,1.0,очень удобный антивирус
7,5,Все устраивает,2017-08-14,1.0,всё устраивать
8,5,"У меня работает все четко. В отличии от банкоматов, которые вечно зависают и тупят",2017-08-14,1.0,я работать всё чётко отличие от банкомат который вечно зависать тупить
9,5,Очень все хорошо👍,2017-08-14,1.0,очень всё хорошо


In [13]:
train_corpus = " ".join(df["Content_1"])
train_corpus = train_corpus.lower()

In [14]:
tokens = word_tokenize(train_corpus)

Отфильтруем данные

и соберём в корпус N наиболее частых токенов

In [15]:
tokens_filtered = [word for word in tokens if word.isalnum()]

In [16]:
from nltk.probability import FreqDist
dist = FreqDist(tokens_filtered)
tokens_filtered_top = [pair[0] for pair in dist.most_common(max_words-1)]

In [17]:
tokens_filtered_top[:10]

['антивирус',
 'всё',
 'не',
 'удобно',
 'очень',
 'работать',
 'на',
 'я',
 'удобный',
 'что']

In [18]:
vocabulary = {v: k for k, v in dict(enumerate(tokens_filtered_top, 1)).items()}

In [19]:
import numpy as np

def text_to_sequence(text, maxlen):
    result = []
    tokens = word_tokenize(text.lower())
    tokens_filtered = [word for word in tokens if word.isalnum()]
    for word in tokens_filtered:
        if word in vocabulary:
            result.append(vocabulary[word])
    padding = [0]*(maxlen-len(result))
    return padding + result[-maxlen:]

In [20]:
X = np.asarray([text_to_sequence(text, max_len) for text in df["Content_1"]], dtype=np.int32)
y = np.asarray(df["class"], dtype=np.int32)

In [21]:
from sklearn.model_selection import  train_test_split

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=55, stratify=y)

In [23]:
X_train[9]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  13,
         6, 224,  21, 168, 208,   3, 254,   4], dtype=int32)

# Keras model

In [24]:
import numpy as np
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Activation, Input, Embedding, Conv1D, GlobalMaxPool1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import TensorBoard
from keras.callbacks import EarlyStopping

In [25]:
num_classes = 2
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [26]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(Conv1D(128, 16))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(16))
model.add(Activation("relu"))
model.add(Dense(num_classes)) #2
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 60, 128)           115200    
                                                                 
 conv1d (Conv1D)             (None, 45, 128)           262272    
                                                                 
 activation (Activation)     (None, 45, 128)           0         
                                                                 
 global_max_pooling1d (Glob  (None, 128)               0         
 alMaxPooling1D)                                                 
                                                                 
 dense (Dense)               (None, 16)                2064      
                                                                 
 activation_1 (Activation)   (None, 16)                0         
                                                        

In [27]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [28]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping])

Epoch 1/30
14/14 [==============================] - 9s 233ms/step - loss: 0.4792 - accuracy: 0.8469 - val_loss: 0.4161 - val_accuracy: 0.8468
Epoch 2/30
14/14 [==============================] - 1s 98ms/step - loss: 0.3720 - accuracy: 0.8582 - val_loss: 0.3162 - val_accuracy: 0.8759
Epoch 3/30
14/14 [==============================] - 1s 65ms/step - loss: 0.2786 - accuracy: 0.8941 - val_loss: 0.2236 - val_accuracy: 0.9120
Epoch 4/30
14/14 [==============================] - 1s 41ms/step - loss: 0.2114 - accuracy: 0.9143 - val_loss: 0.1768 - val_accuracy: 0.9278
Epoch 5/30
14/14 [==============================] - 1s 51ms/step - loss: 0.1769 - accuracy: 0.9251 - val_loss: 0.1600 - val_accuracy: 0.9348
Epoch 6/30
14/14 [==============================] - 1s 69ms/step - loss: 0.1577 - accuracy: 0.9332 - val_loss: 0.1515 - val_accuracy: 0.9386
Epoch 7/30
14/14 [==============================] - 1s 55ms/step - loss: 0.1451 - accuracy: 0.9411 - val_loss: 0.1488 - val_accuracy: 0.9354
Epoch 8/30
1

In [32]:
from sklearn.metrics import f1_score, precision_recall_curve

In [36]:
def metrics(y_test, score):
    metric_dict = {}
    precision, recall, thresholds = precision_recall_curve(y_test, score)
    fscore = (2 * precision * recall) / (precision + recall)
    # locate the index of the largest f score
    ix = np.argmax(fscore)
    metric_dict['thresholds'] = thresholds[ix]
    metric_dict['precision'] = precision[ix]
    metric_dict['recall'] = recall[ix]
    metric_dict['f_score'] = fscore[ix]

    return metric_dict

In [34]:
y_predict = model.predict(X_test, batch_size=batch_size, verbose=1)

y_pr = y_predict[:,:1].flatten()

4/4 [==============================] - 1s 203ms/step


In [37]:
metrics(y_test[:,:1],y_pr)

{'thresholds': 0.36098164,
 'precision': 0.7435158501440923,
 'recall': 0.8528925619834711,
 'f_score': 0.7944572748267897}

In [29]:
import gensim

In [30]:
!wget -q http://vectors.nlpl.eu/repository/20/180.zip
!unzip -q 180.zip


In [47]:
word_vectors = gensim.models.KeyedVectors.load_word2vec_format('/content/model.bin', binary=True).vectors
word_vectors_matrix = [word_vectors[i][:128] for i in range(max_words)]
len(word_vectors_matrix[0])

128

In [48]:

initializer = keras.initializers.Constant(word_vectors_matrix)

model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, embeddings_initializer = initializer, input_length=max_len))
model.add(Conv1D(128, 16))
model.add(Activation("relu"))
model.add(GlobalMaxPool1D())
model.add(Dense(16))
model.add(Activation("relu"))
model.add(Dense(num_classes)) #2
model.add(Activation('sigmoid'))
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 60, 128)           115200    
                                                                 
 conv1d_3 (Conv1D)           (None, 45, 128)           262272    
                                                                 
 activation_9 (Activation)   (None, 45, 128)           0         
                                                                 
 global_max_pooling1d_3 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_6 (Dense)             (None, 16)                2064      
                                                                 
 activation_10 (Activation)  (None, 16)                0         
                                                      

In [49]:
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [50]:
tensorboard=TensorBoard(log_dir='./logs', write_graph=True, write_images=True)
early_stopping=EarlyStopping(monitor='val_loss')


history = model.fit(X_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_split=0.1,
                    callbacks=[tensorboard, early_stopping]
                    )

Epoch 1/30
14/14 [==============================] - 4s 200ms/step - loss: 0.4940 - accuracy: 0.8633 - val_loss: 0.2992 - val_accuracy: 0.8854
Epoch 2/30
14/14 [==============================] - 2s 128ms/step - loss: 0.2754 - accuracy: 0.8924 - val_loss: 0.2551 - val_accuracy: 0.8880
Epoch 3/30
14/14 [==============================] - 1s 100ms/step - loss: 0.2302 - accuracy: 0.9083 - val_loss: 0.2344 - val_accuracy: 0.9038
Epoch 4/30
14/14 [==============================] - 1s 41ms/step - loss: 0.2038 - accuracy: 0.9219 - val_loss: 0.2370 - val_accuracy: 0.9038


In [51]:
y_predict = model.predict(X_test, batch_size=batch_size, verbose=1)

y_pr = y_predict[:,:1].flatten()

4/4 [==============================] - 0s 11ms/step


In [52]:
metrics(y_test[:,:1],y_pr)

{'thresholds': 0.49934426,
 'precision': 0.6489971346704871,
 'recall': 0.7487603305785124,
 'f_score': 0.6953184957789715}

## Выводы
С предообучеными весами обучение занимает меньше времени 8 эпох против 9 На более тяжелых данных разница будет существенней
